In [1]:
# Enable outputting results interactively. 
# All the results from code in a given cell will be displayed instead of only the last one.

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import pickle
import numpy as np
import scipy.stats as stats
import sqlite3
import math
import datetime
import matplotlib.dates as mdates

from re import sub
from decimal import Decimal

import re
import calendar 

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns



In [54]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.ensemble import (GradientBoostingRegressor, RandomForestRegressor)
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from collections import defaultdict

from sklearn.decomposition import PCA

In [4]:
listings_loc =  pickle.load( open( "../data/listings_loc.p", "rb" ) ) 

In [5]:
listings_loc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26502 entries, 0 to 26501
Data columns (total 2 columns):
listing_id    26502 non-null int64
std_nb        26502 non-null object
dtypes: int64(1), object(1)
memory usage: 414.2+ KB


In [6]:
calendar_price =  pickle.load( open( "../data/calendar_pricing.p", "rb" ) ) 
calendar_price.shape

(20595249, 10)

In [7]:
# get standardized neighborhood name

calendar_price = calendar_price.merge(listings_loc, how='left', on = 'listing_id')
calendar_price.shape

(20595249, 11)

In [8]:
# limit to private rooms only

listings_short_avail =  pickle.load( open( "../data/listings_short_avail.p", "rb" ) )
listings_short_avail.room_type.value_counts()
private_room = listings_short_avail[listings_short_avail['room_type'] == "Private room"]
PR_listings = list(set(private_room['id']))

Entire home/apt    112806
Private room        84450
Shared room          6458
Name: room_type, dtype: int64

In [9]:
calendar_price = calendar_price[calendar_price['listing_id'].isin(PR_listings)]

In [10]:
# exclude listings that could be either short-term or long-term rentals


with open('../data/dual_status_listings.pkl', 'rb') as f:
      del_listings = pickle.load(f)
len(del_listings)

2625

In [11]:
# limit to short-term only

calendar_price = calendar_price[~calendar_price['listing_id'].isin(del_listings)]
calendar_price.shape

(8579569, 11)

In [12]:
calendar_price.head(2)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,file_scraped_dt,lead_time,month,std_nb
0,9225,2019-06-04,t,105.0,$105.00,1.0,365.0,2019-06-02,2,2019-06,Potrero Hill
1,9225,2019-06-09,t,105.0,$105.00,1.0,365.0,2019-06-02,7,2019-06,Potrero Hill


In [13]:
calendar_price['listing_id'].nunique()

3647

In [14]:
calendar_price.dtypes

listing_id                  int64
date               datetime64[ns]
available                  object
price                     float64
adjusted_price             object
minimum_nights            float64
maximum_nights            float64
file_scraped_dt    datetime64[ns]
lead_time                   int64
month                      object
std_nb                     object
dtype: object

In [15]:
calendar_price = calendar_price.set_index(pd.DatetimeIndex(calendar_price['date']))

In [16]:
calendar_price['year'] =calendar_price.index.strftime('%Y').astype('int64')

calendar_price['month_yr'] =calendar_price.index.strftime('%m').astype('int64')

calendar_price['month_abbr'] = calendar_price['month_yr'].apply(lambda x: calendar.month_name[x])

calendar_price.drop('month_yr', axis = 1)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,file_scraped_dt,lead_time,month,std_nb,year,month_abbr
date,,,,,,,,,,,,,
2019-06-04,9225,2019-06-04,t,105.0,$105.00,1.0,365.0,2019-06-02,2,2019-06,Potrero Hill,2019,June
2019-06-09,9225,2019-06-09,t,105.0,$105.00,1.0,365.0,2019-06-02,7,2019-06,Potrero Hill,2019,June
2019-06-13,9225,2019-06-13,t,115.0,$115.00,1.0,365.0,2019-06-02,11,2019-06,Potrero Hill,2019,June
2019-06-19,9225,2019-06-19,t,150.0,$150.00,1.0,365.0,2019-06-02,17,2019-06,Potrero Hill,2019,June
2019-06-20,9225,2019-06-20,t,150.0,$150.00,1.0,365.0,2019-06-02,18,2019-06,Potrero Hill,2019,June
2019-06-21,9225,2019-06-21,t,150.0,$150.00,1.0,365.0,2019-06-02,19,2019-06,Potrero Hill,2019,June
2019-06-26,9225,2019-06-26,t,115.0,$115.00,1.0,365.0,2019-06-02,24,2019-06,Potrero Hill,2019,June
2019-06-28,9225,2019-06-28,t,115.0,$115.00,1.0,365.0,2019-06-02,26,2019-06,Potrero Hill,2019,June
2019-07-01,9225,2019-07-01,t,125.0,$125.00,1.0,365.0,2019-06-02,29,2019-06,Potrero Hill,2019,July


In [17]:
calendar_price['scraped_month'] = calendar_price['file_scraped_dt'].dt.month
calendar_price['scraped_month_abbr'] = calendar_price['scraped_month'].apply(lambda x: calendar.month_name[x])
calendar_price.head()

calendar_price = calendar_price.drop('scraped_month', axis = 1)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,file_scraped_dt,lead_time,month,std_nb,year,month_yr,month_abbr,scraped_month,scraped_month_abbr
date,,,,,,,,,,,,,,,,
2019-06-04,9225,2019-06-04,t,105.0,$105.00,1.0,365.0,2019-06-02,2,2019-06,Potrero Hill,2019,6,June,6,June
2019-06-09,9225,2019-06-09,t,105.0,$105.00,1.0,365.0,2019-06-02,7,2019-06,Potrero Hill,2019,6,June,6,June
2019-06-13,9225,2019-06-13,t,115.0,$115.00,1.0,365.0,2019-06-02,11,2019-06,Potrero Hill,2019,6,June,6,June
2019-06-19,9225,2019-06-19,t,150.0,$150.00,1.0,365.0,2019-06-02,17,2019-06,Potrero Hill,2019,6,June,6,June
2019-06-20,9225,2019-06-20,t,150.0,$150.00,1.0,365.0,2019-06-02,18,2019-06,Potrero Hill,2019,6,June,6,June


In [18]:
calendar_price['day_of_week'] = calendar_price['date'].dt.day_name()
calendar_price.day_of_week.value_counts()

Tuesday      1242328
Monday       1239893
Wednesday    1237578
Sunday       1227611
Thursday     1224603
Friday       1205957
Saturday     1201599
Name: day_of_week, dtype: int64

In [19]:
calendar_price['price'].min()

10.0

In [20]:
calendar_price['price'].max()

9589.0

In [21]:
PR_p = np.asarray(calendar_price['price'])

PR_1pct = np.percentile(PR_p, 1)
PR_99pct = np.percentile(PR_p, 99)

PR_1pct, PR_99pct

(50.0, 445.0)

In [44]:
private_room = calendar_price[calendar_price['price'] <= PR_99pct ]
private_room = calendar_price[calendar_price['price'] >= PR_1pct ]

In [45]:
y = np.log(private_room.pop('price'))
y.shape

(8504005,)

In [61]:
private_room.head()

X = private_room[['lead_time', 'std_nb', 'month_abbr','day_of_week']]

,listing_id,date,available,adjusted_price,minimum_nights,maximum_nights,file_scraped_dt,lead_time,month,std_nb,year,month_yr,month_abbr,scraped_month_abbr,day_of_week
date,,,,,,,,,,,,,,,
2019-06-04,9225,2019-06-04,t,$105.00,1.0,365.0,2019-06-02,2,2019-06,Potrero Hill,2019,6,June,June,Tuesday
2019-06-09,9225,2019-06-09,t,$105.00,1.0,365.0,2019-06-02,7,2019-06,Potrero Hill,2019,6,June,June,Sunday
2019-06-13,9225,2019-06-13,t,$115.00,1.0,365.0,2019-06-02,11,2019-06,Potrero Hill,2019,6,June,June,Thursday
2019-06-19,9225,2019-06-19,t,$150.00,1.0,365.0,2019-06-02,17,2019-06,Potrero Hill,2019,6,June,June,Wednesday
2019-06-20,9225,2019-06-20,t,$150.00,1.0,365.0,2019-06-02,18,2019-06,Potrero Hill,2019,6,June,June,Thursday


In [62]:
def get_dummy(df):
    dummy_col=df.select_dtypes('object').columns
    new_df=pd.get_dummies(df, columns=dummy_col, dummy_na=True,prefix=dummy_col)
    return new_df

In [63]:
def tt_split(X, y, test_size=.3):
    X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=test_size)
    return X_train, X_test, y_train, y_test


In [ ]:
X=get_dummy(X)

X_train, X_test, y_train, y_test = tt_split(X,y)

In [ ]:
rfr=RandomForestRegressor(max_depth=5, n_estimators=10)
rfr.fit(X_train.values, y_train)

In [ ]:
print("R2 of test: {:2.2f}".format(rfr.score(X_test, y_test)))

(8504005, 73)